<a href="https://colab.research.google.com/github/john-caballero/Data-Discovery/blob/main/modulo3_manipulacion_datos_faltantes/Handling_Missing_Data_vistoenclase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://posgrado.utec.edu.pe/sites/default/files/2023-08/Testimonial-home-2.jpg" alt="HTML5 Icon" width="900" height="250" >


# **Handling Missing Data**

**Objetivos**

- Aplicar distintas técnicas de imputación:

  - Medidas de tendencia central (media, mediana, moda)

  - Imputación por regresión

  - KNN imputation

  - XGBoost imputador

  - Autoencoders

  - MICE (Multiple Imputation by Chained Equations)

- Entrenar un modelo base (RandomForest o XGBoost) para comparar el impacto de cada estrategia de imputación en términos de performance (accuracy, AUC, etc.)

- Reflexionar sobre las implicancias de cada método para modelos analíticos.

**Dataset**

Loan Prediction - train.csv

Puedes obtenerlo desde:
 https://datahack.analyticsvidhya.com/contest/practice-problem-loan-prediction-iii

Librerías necesarias

In [40]:
%%capture
!pip install pandas numpy seaborn matplotlib scikit-learn xgboost fancyimpute tensorflow missingno

### 1. Carga y diagnóstico inicial

In [41]:
import pandas as pd
import numpy as np

df = pd.read_csv("train.csv")
df.drop("Loan_ID", axis=1, inplace=True)


In [42]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [43]:
df.isnull().sum()

,0
Gender,13
Married,3
Dependents,15
Education,0
Self_Employed,32
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,22
Loan_Amount_Term,14
Credit_History,50


### 2. Medidas de tendencia central (media, mediana, moda)

In [44]:
df_mean = df.copy()


In [53]:
df_mean["LoanAmount"].fillna(df_mean["LoanAmount"].mean(), inplace=True)

In [54]:
df_mean["LoanAmount"].head()

,LoanAmount
0,146.412162
1,128.000000
2,66.000000
3,120.000000
4,141.000000


In [57]:
df_mean["Gender"].fillna('Missing', inplace=True)

In [ ]:
df_mean["Gender"].head(30)

In [ ]:
df_mean["LoanAmount"].fillna(df_mean["LoanAmount"].mean(), inplace=True)
df_mean["Gender"].fillna(df_mean["Gender"].mode()[0], inplace=True)
df_mean["Self_Employed"].fillna(df_mean["Self_Employed"].mode()[0], inplace=True)
df_mean["Dependents"].fillna(df_mean["Dependents"].mode()[0], inplace=True)
df_mean["Credit_History"].fillna(df_mean["Credit_History"].mode()[0], inplace=True)


In [66]:
numeric_features = df_mean.select_dtypes(include='float64').columns.tolist()
df_mean[numeric_features].fillna(df_mean[numeric_features].mean(), inplace=True)

/tmp/ipython-input-66-240499665.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mean[numeric_features].fillna(df_mean[numeric_features].mean(), inplace=True)


### 3. Imputación por regresión lineal

Importamos el modelo de regresión lineal de scikit-learn


In [14]:
from sklearn.linear_model import LinearRegression

Hacemos una copia del dataframe original para no modificarlo directamente


In [15]:
df_reg = df.copy()

In [16]:
# Separamos las filas donde "LoanAmount" NO es nulo (estos datos los usaremos para entrenar el modelo)
train = df_reg[df_reg["LoanAmount"].notnull()]

# Separamos las filas donde "LoanAmount" ES nulo (estos datos se imputarán)
test = df_reg[df_reg["LoanAmount"].isnull()]

Seleccionamos como variables predictoras el ingreso del solicitante y del co-solicitante para el conjunto de entrenamiento


In [17]:
X_train = train[["ApplicantIncome", "CoapplicantIncome"]]

# La variable objetivo a predecir es "LoanAmount"
y_train = train["LoanAmount"]

# Para el conjunto de prueba (filas con "LoanAmount" nulo), también seleccionamos las variables predictoras
X_test = test[["ApplicantIncome", "CoapplicantIncome"]]


Instanciamos y entrenamos el modelo de regresión lineal

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

 el modelo entrenado para predecir los valores faltantes de "LoanAmount"


In [19]:
preds = model.predict(X_test)

Asignamos los valores imputados (predichos) a las posiciones originales donde "LoanAmount" era nulo


In [20]:
df_reg.loc[df_reg["LoanAmount"].isnull(), "LoanAmount"] = preds


### 4. Imputación KNN

Importamos el imputador KNN de scikit-learn
y LabelEncoder para convertir variables categóricas a numéricas



In [21]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

Hacemos una copia del dataframe original para no modificarlo directamente


In [22]:
df_knn = df.copy()

In [23]:
# Aplicamos LabelEncoder a todas las columnas del dataframe.
# Esto convierte variables categóricas (strings) en valores numéricos.
# Nota: KNNImputer solo funciona con variables numéricas.
df_knn = df_knn.apply(LabelEncoder().fit_transform)

In [24]:
# Creamos una instancia del imputador KNN
# n_neighbors=30 indica que se usarán los 30 registros más cercanos (similares)
knn_imputer = KNNImputer(n_neighbors=30)

In [25]:
# Aplicamos el imputador al dataframe y reconstruimos el DataFrame imputado
# fit_transform encuentra los valores faltantes y los imputa usando los vecinos más cercanos
df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(df_knn), columns=df_knn.columns)

### 5. Imputación con XGBoost (modelo para imputar)

In [26]:
# Importamos XGBoost, un modelo de gradient boosting muy potente para tareas de regresión y clasificación
import xgboost as xgb



In [27]:
# Hacemos una copia del DataFrame original para no modificarlo directamente
df_xgb = df.copy()

In [28]:


# Definimos una función para imputar una columna con valores faltantes usando un modelo XGBoost Regressor
def xgb_impute(df, target_col):
    # 1. Separar el conjunto de entrenamiento (valores NO nulos) y prueba (valores nulos) para la variable objetivo
    train = df[df[target_col].notnull()]
    test = df[df[target_col].isnull()]

    # 2. Seleccionar las variables predictoras: excluye la columna objetivo y las columnas completamente vacías
    features = [col for col in df.columns if col != target_col and df[col].notnull().sum() > 0]

    # 3. Elimina registros del entrenamiento que aún tienen missing en las variables predictoras seleccionadas
    train = train.dropna(subset=features)

    # 4. Convierte variables categóricas a dummies (One-Hot Encoding) para entrenamiento
    X_train = pd.get_dummies(train[features])

    # 5. Define la variable objetivo (lo que vamos a imputar)
    y_train = train[target_col]

    # 6. Convierte también el conjunto de prueba a dummies
    X_test = pd.get_dummies(test[features])

    # 7. Asegura que X_test tenga las mismas columnas que X_train (rellena columnas faltantes con 0)
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 8. Entrena el modelo XGBoost con 100 árboles
    model = xgb.XGBRegressor(n_estimators=100)
    model.fit(X_train, y_train)

    # 9. Predice los valores faltantes en la variable objetivo
    preds = model.predict(X_test)

    # 10. Asigna los valores imputados a las posiciones originales con missing
    df.loc[df[target_col].isnull(), target_col] = preds

    # 11. Devuelve el DataFrame imputado
    return df

# Aplicamos la función para imputar los valores faltantes en la variable "LoanAmount"
df_xgb = xgb_impute(df_xgb, "LoanAmount")


### 6. Imputación MICE (Multivariate Imputation)

In [29]:
# Importamos IterativeImputer desde fancyimpute (implementa el algoritmo MICE)
from fancyimpute import IterativeImputer


In [30]:
# Hacemos una copia del DataFrame original para no modificarlo directamente
df_mice = df.copy()

In [31]:

# Aplicamos LabelEncoder a todas las columnas para convertir variables categóricas a numéricas
# Esto es necesario porque MICE solo puede trabajar con variables numéricas
df_mice = df_mice.apply(LabelEncoder().fit_transform)

# Creamos una instancia del imputador MICE (IterativeImputer)
# Este imputador usa regresiones iterativas entre variables para predecir valores faltantes
mice = IterativeImputer()

# Aplicamos el imputador a los datos
# fit_transform entrena los modelos internamente y realiza la imputación
df_mice_imputed = pd.DataFrame(mice.fit_transform(df_mice), columns=df_mice.columns)


### 7. Autoencoders para imputación

In [32]:
# Importamos TensorFlow y los componentes necesarios para construir una red neuronal (autoencoder)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Hacemos una copia del DataFrame original para no modificarlo directamente
df_auto = df.copy()

# Convertimos todas las variables categóricas a numéricas con LabelEncoder
df_auto = df_auto.apply(LabelEncoder().fit_transform)

# Imputamos valores faltantes inicialmente con la media de cada columna
# Esto es necesario porque las redes neuronales no pueden recibir NaNs como entrada
df_auto = df_auto.fillna(df_auto.mean())

# Convertimos el DataFrame a un array de NumPy para entrenar la red
X = df_auto.values


In [ ]:
# Definimos la arquitectura del Autoencoder (una red neuronal simétrica)
model = Sequential([
    Dense(32, activation='relu', input_shape=(X.shape[1],)),  # Capa de entrada → 32 neuronas
    Dense(16, activation='relu'),                              # Capa oculta comprimida (bottleneck)
    Dense(32, activation='relu'),                              # Capa de expansión (simétrica)
    Dense(X.shape[1])                                          # Capa de salida (reconstrucción completa)
])


In [ ]:
# Compilamos el modelo usando el optimizador Adam y la pérdida de error cuadrático medio (MSE)
model.compile(optimizer='adam', loss='mse')

# Entrenamos el modelo para que aprenda a reconstruir sus propias entradas
# El autoencoder aprende patrones en los datos para luego usarlos en la imputación
model.fit(X, X, epochs=100, batch_size=16, verbose=0)


In [ ]:
# Usamos el modelo entrenado para predecir la versión reconstruida de los datos
# Esta salida será utilizada como imputación final
X_pred = model.predict(X)

# Convertimos la salida a un DataFrame con los mismos nombres de columnas que el original
df_auto_imputed = pd.DataFrame(X_pred, columns=df_auto.columns)


### 8. Evaluación del impacto predictivo

Utilizaremos un modelo de clasificación (XGBClassifier) para predecir Loan_Status con los diferentes datasets imputados.

In [36]:
# Importamos herramientas
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier


In [37]:
# Definimos una función que evalúa un DataFrame ya imputado
# Entrena un modelo para predecir "Loan_Status" y calcula métricas de desempeño
def evaluate_model(df, name=""):
    df = df.copy()  # Hacemos copia para no alterar el original

    # Eliminamos cualquier fila que aún tenga valores faltantes
    df = df.dropna()

    # Separamos variables predictoras (X) de la variable objetivo (y)
    X = df.drop("Loan_Status", axis=1)
    y = df["Loan_Status"]

    # Si la variable objetivo es categórica (tipo object), la codificamos numéricamente
    if y.dtype == 'O':
        y = LabelEncoder().fit_transform(y)

    # Convertimos todas las variables categóricas (en X) a variables dummy (One-Hot Encoding)
    X = pd.get_dummies(X)

    # Dividimos los datos en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Creamos y entrenamos el clasificador XGBoost
    # Desactivamos use_label_encoder y usamos logloss como métrica
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    # Generamos predicciones de clase y de probabilidad
    preds = model.predict(X_test)
    probas = model.predict_proba(X_test)[:,1]  # Probabilidad de clase positiva

    # Mostramos métricas: Accuracy y AUC
    print(f"Evaluación con {name}")
    print(f"Accuracy: {accuracy_score(y_test, preds):.4f}")
    print(f"AUC: {roc_auc_score(y_test, probas):.4f}\n")


In [ ]:
# Evaluamos el desempeño del modelo con diferentes versiones del dataset imputado
evaluate_model(df_mean, "Media/Moda")               # Imputación simple
evaluate_model(df_reg, "Regresión")                 # Imputación por regresión lineal
evaluate_model(df_knn_imputed, "KNN")               # Imputación con K-Nearest Neighbors
evaluate_model(df_xgb, "XGBoost")                   # Imputación supervisada con XGBoost
evaluate_model(df_mice_imputed, "MICE")             # Imputación iterativa por MICE
df_auto_imputed["Loan_Status"] = df["Loan_Status"]
evaluate_model(df_auto_imputed, "Autoencoder")      # Imputación usando autoencoders


### Preguntas para reflexión

- ¿Qué técnica ofrece mejor balance entre simplicidad y precisión?

- ¿Qué riesgos podría implicar usar una técnica muy compleja como Autoencoders o XGBoost para imputar?

- ¿Por qué KNN puede ser sensible a la escala de los datos o a outliers?

- ¿Cómo se podría incorporar la incertidumbre de la imputación en el modelo final?

- ¿Qué tipo de datos (categóricos, numéricos, multivariados) favorecen el uso de MICE sobre otros métodos?

### Conclusión

La imputación no es solo un paso técnico: es una decisión analítica que puede alterar el resultado del modelo. Evaluar diferentes métodos no solo mejora el desempeño, sino también la confianza en los modelos desarrollados. Entender cuándo usar cada técnica y su impacto es clave para un análisis responsable.

---

# Gracias por completar este laboratorio!

---
